<a href="https://colab.research.google.com/github/saeidsaadatigero/ai-app/blob/main/ai_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install flask flask-ngrok transformers pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 67.2 MB/s eta 0:00:00


In [ ]:
!pip install pyngrok

In [ ]:
!wget -qO- https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip | unzip -
!./ngrok authtoken YOUR_NGROK_AUTH_TOKEN

UnZip 6.00 of 20 April 2009, by Debian. Original by Info-ZIP.

Usage: unzip [-Z] [-opts[modifiers]] file[.zip] [list] [-x xlist] [-d exdir]
  Default action is to extract files in list, except those in xlist, to exdir;
  file[.zip] may be a wildcard.  -Z => ZipInfo mode ("unzip -Z" for usage).

  -p  extract files to pipe, no messages     -l  list files (short format)
  -f  freshen existing files, create none    -t  test compressed archive data
  -u  update files, create if necessary      -z  display archive comment only
  -v  list verbosely/show version info       -T  timestamp archive to latest
  -x  exclude files that follow (in xlist)   -d  extract files into exdir
modifiers:
  -n  never overwrite existing files         -q  quiet mode (-qq => quieter)
  -o  overwrite files WITHOUT prompting      -a  auto-convert any text files
  -j  junk paths (do not make directories)   -aa treat ALL files as text
  -U  use escapes for all non-ASCII Unicode  -UU ignore any Unicode fields
  -C  mat

In [ ]:
!./ngrok http 5000

/bin/bash: line 1: ./ngrok: No such file or directory


In [ ]:
!pip install flask_cors

In [15]:
from flask import Flask, request, jsonify
from pyngrok import ngrok
from transformers import pipeline
import fitz  # PyMuPDF
from flask_cors import CORS


# بارگذاری مدل
model_name_or_path = "m3hrdadfi/xlmr-large-qa-fa"
nlp = pipeline('question-answering', model=model_name_or_path, tokenizer=model_name_or_path)




# تعریف Flask App
app = Flask(__name__)
CORS(app)  # اضافه کردن CORS

# تنظیم ngrok با استفاده از توکن
ngrok.set_auth_token("2q4tkDIv5klP6mmWQ6Ecr7f3GXc_7EHvGE6tf51FH1pLBz4do")
public_url = ngrok.connect(5000)
print(f"Public URL: {public_url}")

# مسیر پیش‌فرض
@app.route("/", methods=["GET", "POST"])
def index():
    return "Flask is running!"

# تابع برای استخراج متن از PDF
def extract_text_from_pdf(file_path):
    pdf_document = fitz.open(file_path)
    text = ""
    for page_num in range(len(pdf_document)):
        page = pdf_document.load_page(page_num)
        text += page.get_text("text")
    pdf_document.close()
    return text

# مسیر آپلود و پردازش فایل
@app.route('/upload', methods=['POST'])
def upload_and_ask():
    # دریافت فایل و سؤال
    file = request.files['file']
    question = request.form['question']

    # ذخیره فایل PDF
    file_path = "uploaded.pdf"
    file.save(file_path)

    # استخراج متن از PDF
    context = extract_text_from_pdf(file_path)
    if not context.strip():
        return jsonify({"error": "متن PDF استخراج نشد."}), 400

    # پردازش سؤال و پاسخ
    result = nlp(question=question, context=context)
    answer = result["answer"].strip()

    return jsonify({"question": question, "answer": answer})

if __name__ == "__main__":
    app.run(port=5000)


Public URL: NgrokTunnel: "https://ee76-35-231-11-103.ngrok-free.app" -> "http://localhost:5000"
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:54:14] "OPTIONS /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:54:54] "OPTIONS /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:54:56] "OPTIONS /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:54:56] "OPTIONS /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:54:56] "OPTIONS /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:54:56] "OPTIONS /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:55:46] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:58:23] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:58:33] "POST /upload HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [11/Dec/2024 16:58:34